# Planning observations with `astroplan`

In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
import pytz

from astroplan import Observer, FixedTarget

FileNotFoundError: [Errno 2] No such file or directory: '/home/user/.astropy/cache/download/py3/42dfee0e804ac4ccd2a4e7ae0db2f6ce'

# Time and Dates
- ### All dates and times in are UTC: *Coordinated Universal Time* .
- All `Time` calculation assume that the time is UTC.
- UTC is related to  Greenwich Mean Time (GMT) but does not change with a change of seasons.
- Time will default to 00:00:00 UTC.

In [0]:
my_date1 = Time("2018-02-12 13:40:15")

my_date1

In [0]:
my_date1.value

In [0]:
print(my_date1)

In [0]:
my_date2 = Time("2018-02-12")

my_date2

### Current UTC Time

In [0]:
current_time = Time.now()    # Current UTC Time

current_time

In [0]:
print("The current time is {0}".format(current_time))

### Different Date Formats

In [0]:
print("The current Julian Date is {0:.2f}".format(current_time.jd))

print("The current Modified Julian Date is {0:.2f}".format(current_time.mjd))

print("The current unix Epoch is {0:.2f}".format(current_time.unix))  # Seconds since (Jan 01, 1970 00:00:00 UTC)

print("The current fraction of a year is {0:.2f}".format(current_time.decimalyear))

### Math with Time and Dates

In [0]:
print("In 1 hour and 25 minutes it will be {0} UTC".format(current_time + 1*u.h + 25*u.min))

In [0]:
Christmas = Time("2018-12-25 00:00:00")

delta_xmas = Christmas - current_time

print("It is {0:.2f} until christmas, which is {1:.2f}, or {2:.2f}"
      .format(delta_xmas.to(u.d),delta_xmas.to(u.fortnight),delta_xmas.to(u.s)))

### Working with timezones (local time)

- The python package `pytz` is used to try to deal with local timezones
- [Timezone List](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones)
- Only use timezone conversions for printouts, NEVER calculations!
- Working with tomezones is a [quick path to maddness!](https://xkcd.com/1883/)

In [0]:
my_timezone = pytz.timezone('US/Pacific')

local_now = current_time.to_datetime(my_timezone)

In [0]:
print("The current local time is {0}".format(local_now))

---

### [Accurate Time](http://bmmorris.blogspot.com/2015/06/ut1-utc-and-astropy.html) - `UT1`

`AstroPy` calculates the times of events to a very high accuracy. To do this, is has to account for the fact that  Earth's rotation period is constantly changing due to tidal forces and changes in the Earth's moment of inertia.

To do this, `AstroPy` uses a time convention called `UT1`. This system is tied to the rotation of the Earth with repect to the positions of distant quasars. Since the Earth's rotation is constantly changing, the time system `UT1` is constanly changing with repect to `UTC`. 

The orientation of the Earth, which must be measured continuously to keep `UT1` accurate. This measurement is logged by the International Earth Rotation and Reference Systems Service (IERS). They publish a "bulletin" with the most recent measurements of the Earth's orientation. This bulletin is constantly being updated.

You will run into occasions when you will get a warning that your dates are out of range of the IERS bulletin. To update the bulletin, run the follow block of code:

---

In [0]:
from astroplan import download_IERS_A

download_IERS_A()

# Places

## Setting your location - `Observer`

In [0]:
astrolab = Observer(longitude = -122.311473 * u.deg,
                    latitude = 47 * u.deg + 39 * u.arcmin + 15 * u.arcsec,
                    elevation = 63.4 * u.m,
                    timezone = 'US/Pacific',
                    name = "Astrolab"
                    )

In [0]:
astrolab

In [0]:
astrolab.timezone

In [0]:
local_now = current_time.to_datetime(astrolab.timezone)
print(local_now)

### Information at your location

In [0]:
sunset_here = astrolab.sun_set_time(current_time, which='nearest')

print("Sunset will be at {0.iso} UTC".format(sunset_here))

In [0]:
print("Sunset will be at {0} local time".format(sunset_here.to_datetime(astrolab.timezone)))

### MRO

The [Manastash Ridge Observatory (MRO)](https://sites.google.com/a/uw.edu/mro/) is operated by the Astronomy Department of the University of Washington, for the training of graduate and undergraduate students, as well as for astronomical research.

In [0]:
mro = Observer.at_site('mro')  # many observatories are built-in to astroplan

mro

In [0]:
mro.timezone = pytz.timezone('US/Pacific')

In [0]:
mro

In [0]:
sunset_mro = mro.sun_set_time(current_time, which='nearest')

print("Sunset at MRO will be at {0} local time".format(sunset_mro.to_datetime(mro.timezone)))

In [0]:
(sunset_here - sunset_mro).to(u.min)

### Local Siderial Time (LST) will tell you the Right Ascension on the meridian right now.

- You can use a [star chart](./Astro_Coordinates.pdf) to find what constellations are visible now.

In [0]:
midnight_mro = mro.midnight(current_time, which='next')

mro.local_sidereal_time(midnight_mro)

## When can you observe?

#### set up a reference time:

In [0]:
reference_time = current_time

### Astronomical twilight is when the Sun is 18 degrees below the horizon

* observation begin at `twilight_evening_astronomical`
* observations end at `twilight_moring_astronomical`

You can choose which `twilight_evening_astronomical` relative to the `current_time` would you like to calculate:

* `next`
* `previous`
* `nearest`

In [0]:
astro_set = mro.twilight_evening_astronomical(reference_time, which='nearest')
astro_rise = mro.twilight_morning_astronomical(reference_time, which='next')

print("Astronomical Evening Twilight starts at {0.iso} UTC".format(astro_set))
print("Astronomical Midnight is at {0.iso} UTC".format(midnight_mro))
print("Astronomical Morning Twilight starts at {0.iso} UTC".format(astro_rise))

In [0]:
observing_length = (astro_rise - astro_set).to(u.h)

print("You can observe for {0:.1f} at MRO tonight".format(observing_length))

In [0]:
# Local Times

print("Tonight's observing at MRO starts at {0},\n peaks at {1} and,\n ends at {2} local time"
      .format(astro_set.to_datetime(mro.timezone),
              midnight_mro.to_datetime(mro.timezone),
              astro_rise.to_datetime(mro.timezone)))

# Things

## Objects in the sky - `FixedTarget`

### You can define targets by [coordinates](./Astro_Coordinates.pdf)

In [0]:
coords = SkyCoord('02h19m00.0s', '+57d07m042s', frame='icrs')

ngc869 = FixedTarget(name='NGC869', coord=coords)

In [0]:
ngc869.ra

In [0]:
ngc869.ra.hms

### Can you see the object?

In [0]:
mro.target_is_up(midnight_mro, ngc869)

### Where can you see it a midnight?

In [0]:
where_to_look = mro.altaz(midnight_mro, ngc869)

In [0]:
where_to_look.alt

In [0]:
where_to_look.az

### Most targets can be defined by name

In [0]:
my_target = FixedTarget.from_name("ngc2403")

In [0]:
my_target.coord 

In [0]:
my_target.ra.hms

## Objects in the sky - Moving Targets (solar system targets)

- `Astropy` used the `jplephem` package to calculate the positions
- The built-in solar system objects are: 'sun', 'mercury', 'venus', 'earth-moon-barycenter', 'earth', 'moon', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune', 'pluto'

In [0]:
from astropy.coordinates import get_sun, get_body, get_moon
from astroplan import moon_illumination

### Is the Sun above the horizion now?

In [0]:
sun_now = get_body('sun',current_time)

sun_now

In [0]:
sun_location = astrolab.altaz(current_time, sun_now)

sun_location.alt, sun_location.az

In [0]:
moon_now = get_body('moon',current_time)

In [0]:
moon_illumination(current_time)

In [0]:
sun_now.separation(moon_now)

# Planning - Observing at MRO

### [Air Mass](https://en.wikipedia.org/wiki/Air_mass_%28astronomy%29) is the optical path length through Earth’s atmosphere. 

* At sea-level, the air mass at the zenith is 1. 
* Air mass increases as you move toward the horizon.
* Air mass at the horizon is approximately 38.
* The best time to observe a target is at minimum airmass.
* When the airmass of your target is getting close to 2, you should be observing another target.

In [0]:
my_target = FixedTarget.from_name("ngc2403")

In [0]:
mro.target_is_up(midnight_mro, my_target)

### Object is up at midnight at MRO - good

In [0]:
altaz_my_target = astrolab.altaz(midnight_mro, my_target)

altaz_my_target.alt, altaz_my_target.az

### Nice high altitude - looking good

In [0]:
# You can find the airmass by using the .secz method

altaz_my_target.secz

### Airmass < 2, you are good to go.

## Planning observation is easier with plots

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

from astroplan import time_grid_from_range

from astroplan.plots import plot_sky, plot_airmass

In [0]:
plot_sky(my_target, mro, midnight_mro);

In [0]:
start_time = astro_set
end_time = astro_rise

observing_range = [astro_set, astro_rise]

time_grid = time_grid_from_range(observing_range)

plot_sky(my_target, mro, time_grid);

### Plot the airmass of the target over the night

In [0]:
plot_airmass(my_target, mro, time_grid);

#### This is good target for observation at MRO for most of the night

### Not all targets can (or should) be observed at all locations

In [0]:
low_target = FixedTarget.from_name("Sirius")

In [0]:
mro.target_is_up(midnight_mro, low_target)

In [0]:
plot_sky(low_target, mro, time_grid);

In [0]:
plot_airmass(low_target, mro, time_grid, max_airmass=5.0);

#### Not looking good

As you can see, this is bad target for observation at MRO.

### Finder Charts - (Warning: This may not always work depending on the Skyview website)

In [0]:
from astroplan.plots import plot_finder_image
from astroquery.skyview import SkyView

In [0]:
plot_finder_image(my_target)

In [0]:
# plot_finder_image defaults to a field of view of 10 u*arcmin
# You can specify a different fov

plot_finder_image(my_target, fov_radius= 30 * u.arcmin)